<a href="https://colab.research.google.com/github/braingeneers/experiment/blob/master/src/aws/OrganoidExperiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <blue> Create Your Organoid Experiment </blue> 



### Import helper python libraries

In [0]:
from google.colab import files
import sys
import os
import socket
import pickle
import random
import time
from datetime import datetime
import pytz #convert between timezones
from pytz import timezone
import json
import uuid #generate guids
import numpy as np
import datetime



## 1) Set Experiment Parameters



In [0]:
#@title Experiment Parameters { run: "auto", vertical-output: true }

user_email = 'kvoitiuk@ucsc.edu' #@param {type:"string"}

#@markdown <br> Would you like to use an organoid that is simulated or real?
organoid_type = 'real' #@param ["simulated", "real"]

#@markdown <br> Would you like to configure your experiment beforehand, or perform it in interactive (dynamic) mode?
organoid_input = 'configured' #@param ["configured", "dynamic"]

#@markdown ### For dynamic experiments only:
#@markdown Please enter the IP address and port of the machine running your interactive program:
client_ip = "" #@param {type:"string"}
client_port =  '5006' #@param {type:"string"}

current_ip = str(socket.gethostbyname(socket.gethostname()))
print("To use this machine, the IP is:", current_ip)

To use this machine, the IP is: 172.28.0.2


## 2) Auto-Generate Experiment GUID

In [0]:
# Generate the GUID
e_guid = str(uuid.uuid4())

# Display the GUID
print("Experiment guid: ", e_guid)

Experiment guid:  eb5e65ef-fc38-4687-87b2-56dbc44baf3a


## 3) Auto-Create Experiment JSON
This JSON file will be used to request an experiment with your desired parameters


In [0]:
# Current time in UTC
now_utc = datetime.datetime.now(timezone('UTC'))

# Convert to US/Pacific time zone
now_pacific = now_utc.astimezone(timezone('US/Pacific'))

# Specify date formatting
fmt = "%H:%M:%S %d-%m-%Y %Z%z" 

# Make date timestamp
localdate_string = now_pacific.strftime(fmt)


# Put JSON together
data = {
	"experiment": {
		"guid": e_guid,
		"email": user_email,
		"type": organoid_type,
		"input": organoid_input,
		"date": localdate_string
}}

# For dynamic experiment, add two more fields
if(organoid_input == "dynamic"):
  data["experiment"]["client_ip"] = client_ip
  data["experiment"]["client_port"] = client_port
  
  
# Save JSON file
with open(e_guid + ".json", 'w') as fp:
		json.dump(data, fp)
print("Saved!\n")
    
# Display JSON
print(json.dumps(data, indent=4))
  


Saved!

{
    "experiment": {
        "guid": "eb5e65ef-fc38-4687-87b2-56dbc44baf3a",
        "email": "kvoitiuk@ucsc.edu",
        "type": "real",
        "input": "configured",
        "date": "14:29:07 05-03-2019 PST-0800"
    }
}


## 3) Create Stimulation File
For configured experiments, please create your numpy stimulation file. <br>
Optional for dynamic experiment.

In [0]:
# Edit stimulation patterns as: [pattern (0-255), duration (ms)]	
x = np.array([[255, 10], [1, 10], [2, 10], [4, 10], [8, 10], [16, 10], [32, 10], [64, 10], [128, 10], [0, 10], [1, 10], [2, 10], [4, 10], [8, 10], [16, 10], [32, 10], [64, 10], [128, 10], [0, 10], [255, 100]])
	
# Name and save numpy file
f = e_guid + ".npy"
np.save(f, x)
print("Saved!")

Saved!


## 4) Download Your Experiment Files
...and submit them to the portal: https://www.braingeneers.org/receive/

In [0]:
# Download JSON
files.download(e_guid + ".json")

# If numpy file was created, download numpy
if (os.path.isfile(e_guid + ".npy")):
   files.download(e_guid + ".npy")